In [1]:
from sklearn.ensemble import RandomForestClassifier as RFC_
from sklearn.ensemble import RandomForestRegressor as RFR_

In [2]:
import time
ini_time=time.time()

import datetime

import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np




#%run 'prev_comex_ncm.ipynb'

# Define parametros
cat_cols=['CO_MES','nick','SG_UF_NCM','CO_PAIS','CO_URF']
l=20

comex_ncm=pd.read_csv('~/git/BD/temp/imp_fert_quanti.csv').sort_values(by=['CO_ANO','CO_MES'])

# Filtra top
top_nick=comex_ncm.groupby('nick').KG_LIQUIDO.sum().sort_values().index[-2:]
top_uf=comex_ncm.groupby('SG_UF_NCM').KG_LIQUIDO.sum().sort_values().index[-2:]
top_pais=comex_ncm.groupby('CO_PAIS').KG_LIQUIDO.sum().sort_values().index[-2:]
top_urf=comex_ncm.groupby('CO_URF').KG_LIQUIDO.sum().sort_values().index[-2:]
comex_ncm=comex_ncm[(comex_ncm.nick.isin(top_nick))&(comex_ncm.SG_UF_NCM.isin(top_uf))&(comex_ncm.CO_PAIS.isin(top_pais))&(comex_ncm.CO_URF.isin(top_urf))]

# Salva datas de início e fim
comex_ncm['date']=pd.to_datetime(comex_ncm.CO_ANO.astype(str)+'/'+comex_ncm.CO_MES.astype(str)+'/1')
date_ini=comex_ncm.date.max()
date_fim=comex_ncm.date.max()

In [3]:
# Sparse Matrix 
# Matriz 1
anos=pd.DataFrame({'CO_ANO':np.arange(comex_ncm.CO_ANO.max(),comex_ncm.CO_ANO.max()+1),'key':0})
# Matriz 2
cat=comex_ncm[cat_cols].drop_duplicates().assign(key=0)
# Multiplica
c_c=anos.merge(cat,how='outer').drop(columns='key')        
comex_ncm=c_c.merge(comex_ncm,how='outer').fillna(0)
# Refaz data
comex_ncm['date']=pd.to_datetime(comex_ncm.CO_ANO.astype(str)+'/'+comex_ncm.CO_MES.astype(str)+'/1')
# Apara as bordas
comex_ncm=comex_ncm[(comex_ncm.date<=date_fim)&(comex_ncm.date>=date_ini)]

In [4]:
comex_ncm

,CO_ANO,CO_MES,nick,SG_UF_NCM,CO_PAIS,CO_URF,KG_LIQUIDO,date
573,2024,4,MOP_k,PR,676,917800,52606851.0,2024-04-01
574,2024,4,MOP_k,PR,999,917800,16800000.0,2024-04-01
575,2024,4,UREIA_k,PR,676,817800,0.0,2024-04-01
576,2024,4,UREIA_k,PR,676,917800,6524000.0,2024-04-01
577,2024,4,UREIA_k,PR,999,917800,22839957.0,2024-04-01
578,2024,4,UREIA_k,RS,676,917800,0.0,2024-04-01
579,2024,4,UREIA_k,RS,999,917800,800000.0,2024-04-01


In [5]:
target_cols=['oc','KG_LIQUIDO']
cat_cols=['CO_MES','nick','SG_UF_NCM','CO_PAIS','CO_URF']
md_c=15
md_r=1

# Identifica as datas de previsao
pred_max=comex_ncm.date.max()
pred_max=comex_ncm.date.max()
# Classificação ##########################################################################################

# Marca ocorrências
comex_ncm=comex_ncm.assign(oc=0)
comex_ncm.loc[comex_ncm[target_cols[1]]>0,'oc']=1
comex_ncm

,CO_ANO,CO_MES,nick,SG_UF_NCM,CO_PAIS,CO_URF,KG_LIQUIDO,date,oc
573,2024,4,MOP_k,PR,676,917800,52606851.0,2024-04-01,1
574,2024,4,MOP_k,PR,999,917800,16800000.0,2024-04-01,1
575,2024,4,UREIA_k,PR,676,817800,0.0,2024-04-01,0
576,2024,4,UREIA_k,PR,676,917800,6524000.0,2024-04-01,1
577,2024,4,UREIA_k,PR,999,917800,22839957.0,2024-04-01,1
578,2024,4,UREIA_k,RS,676,917800,0.0,2024-04-01,0
579,2024,4,UREIA_k,RS,999,917800,800000.0,2024-04-01,1


In [6]:
# Codifica categorias
comex_ncm[cat_cols]=comex_ncm[cat_cols].astype('category')    
comex_ncm_dumm=pd.get_dummies(comex_ncm, prefix_sep='~')

# Data em formato para processamento
comex_ncm_dumm.date=comex_ncm_dumm.date.astype(int)
# Ano substituido por date, que da a sequencia dos meses
comex_ncm_dumm.drop(columns='CO_ANO',inplace=True)

comex_ncm_dumm

,KG_LIQUIDO,date,oc,CO_MES~4,nick~MOP_k,nick~UREIA_k,SG_UF_NCM~PR,SG_UF_NCM~RS,CO_PAIS~676,CO_PAIS~999,CO_URF~817800,CO_URF~917800
573,52606851.0,1711929600000000000,1,True,True,False,True,False,True,False,False,True
574,16800000.0,1711929600000000000,1,True,True,False,True,False,False,True,False,True
575,0.0,1711929600000000000,0,True,False,True,True,False,True,False,True,False
576,6524000.0,1711929600000000000,1,True,False,True,True,False,True,False,False,True
577,22839957.0,1711929600000000000,1,True,False,True,True,False,False,True,False,True
578,0.0,1711929600000000000,0,True,False,True,False,True,True,False,False,True
579,800000.0,1711929600000000000,1,True,False,True,False,True,False,True,False,True


In [7]:
# Separa train, test_max e test_max
def train_test_max_test_max(df, pred_max, pred_max):
    '''Identifica train, test_max, test_max'''
    
    train=df[(df.date.astype('datetime64[ns]')>pred_max)&(df.date.astype('datetime64[ns]')<pred_max)]
    # Identifica test na data mínima e máxima
    test_max=df[df.date.astype('datetime64[ns]')==pred_max]
    test_max=df[df.date.astype('datetime64[ns]')==pred_max]    
    return [train,test_max,test_max]
[train_c,test_max_c,test_max_c]=train_test_max_test_max(comex_ncm_dumm,pred_max, pred_max)

SyntaxError: duplicate argument 'pred_max' in function definition (784801267.py, line 2)

In [ ]:
# Separa em X e y
def Xy(train,test_max, test_max, col_target,col_del):
    X_train=train.drop(columns=[col_target]+[col_del])
    y_train=train[col_target]
    X_test_max=test_max.drop(columns=[col_target]+[col_del])
    y_test_max=test_max[col_target]
    X_test_max=test_max.drop(columns=[col_target]+[col_del])
    y_test_max=test_max[col_target]
    return [X_train,y_train,X_test_max,y_test_max,X_test_max,y_test_max]

[X_train_c,y_train_c,X_test_max_c,y_test_max_c,X_test_max_c,y_test_max_c]=Xy(train_c,test_max_c,test_max_c,target_cols[0],target_cols[1])
# Prediz

In [ ]:
def fit_pred(RF,md,X_train,y_train,X_test_max,y_test_max,X_test_max,y_test_max, col_target):
    # Fit
    RF=RF(max_depth=md)
    RF.fit(X_train,y_train)

    # Prediz Max
    test_pred_max=X_test_max.assign(col1=RF.predict(X_test_max)).rename(columns={'col1':'str(col_target)+'_pred''})
    test_pred_max=test_pred_max.assign(col2=y_test_max).rename(columns={`'col2':(str(col_target)+'_pred')})

    # Prediz min
    test_pred_min=X_test_min.assign(col1=RF.predict(X_test_min)).rename(columns={'col1':'str(col_target)+'_pred''})
    test_pred_min=test_pred_min.assign(col2=y_test_min).rename(columns={`'col2':(str(col_target)+'_pred')})

    return [test_pred_max,test_pred_max]

In [ ]:
list_test_pred_c=fit_pred(RFC_,md_c,X_train_c,y_train_c,X_test_max_c,y_test_max_c,X_test_max_c,y_test_max_c,target_cols[0])

test_pred_max_c,test_pred_max_c=list_test_pred_c

In [ ]:
# Regressão ##########################################################################################

# Filtra deixando passar somente ocorridos e previstos de ocorrer
train_r=train_c[train_c.oc==1]
test_max_r=test_pred_max_c[(test_pred_max_c.oc==1)&(test_pred_max_c.oc_pred==1)]
test_max_r=test_pred_max_c[(test_pred_max_c.oc==1)&(test_pred_max_c.oc_pred==1)]

In [ ]:
#if (len(train_r)>0) & ((len(test_max_r)>0) | (len(test_max_r)>0)):

# Filtra deixando passar para o treino somente combinacoes categoricas presentes na predicao
def decode(df):
    df=pd.concat([
    df.select_dtypes(exclude='bool'),
    pd.from_dummies(df.select_dtypes(include='bool'),sep='~')],axis=1)
    return df
train_r=decode(train_r)
test_max_r=decode(test_max_r)
test_max_r=decode(test_max_r)

In [ ]:
filtro=pd.concat([test_max_r,test_max_r])[cat_cols].drop_duplicates()
train_r=train_r.merge(filtro)
train_r.head(1)

In [ ]:
comex_ncm.date=comex_ncm.date.astype('int')
test_max_r=test_max_r.merge(comex_ncm)
test_max_r=test_max_r.merge(comex_ncm)

In [ ]:
test_max_r

In [ ]:
comex_ncm_r=pd.concat([train_r,test_max_r,test_max_r])
comex_ncm_r

In [ ]:
# Codifica categorias
comex_ncm[cat_cols]=comex_ncm[cat_cols].astype('category')    
comex_ncm_dumm=pd.get_dummies(comex_ncm, prefix_sep='~')

comex_ncm_dumm.date=comex_ncm_dumm.date.astype(int)

# Separa train and tests
[train_r,test_max_r,test_max_r]=train_test_max_test_max(comex_ncm_dumm,pred_max, pred_max)

# Separa X e y
[X_train_r,y_train_r,X_test_max_r,y_test_max_r,X_test_max_r,y_test_max_r]=Xy(train_r,test_max_r,test_max_r,target_cols[1],target_cols[0])

# Prediz
list_test_pred_r=fit_pred(RFR_,md_r,X_train_r,y_train_r,X_test_max_r,y_test_max_r,X_test_max_r,y_test_max_r,target_cols[1])

test_pred_max_r,test_pred_max_r=list_test_pred_r

In [ ]:
decode(X_test_max_r)

In [ ]:
test_pred_max_r

In [ ]:
#else:
#    test_pred_max_r,test_pred_max_r=pd.DataFrame(), pd.DataFrame()

# Junta classi e regress resultados
test_pred_c=pd.concat([test_pred_max_c,test_pred_max_c])
test_pred_r=pd.concat([test_pred_max_r,test_pred_max_r])
test_pred=test_pred_c.merge(test_pred_r)

In [ ]:
test_pred=decode(test_pred)
                     
test_pred

In [ ]:
print("preve_comex_ncm(comex_ncm,target_cols=['oc','KG_LIQUIDO'],cat_cols=['CO_MES','nick','SG_UF_NCM','CO_PAIS','CO_URF'],md_c=15,md_r=1):")

In [ ]:
print("RandomForestClassifier as RFC_ & RandomForestRegressor as RFR_")